In [1]:
import os
import pandas
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
wic_path  = "/content/drive/My Drive/datasets/WiC_dataset"
patht = "/content/drive/My Drive/datasets/WiC_dataset"
path_wic_constant = "/content/drive/My Drive/datasets/WiC_dataset"

In [4]:
! ls "/content/drive/My Drive/datasets/Split_WiC_dataset"

Cosine_Sim  dev  train


In [5]:
wic_train_path_d = os.path.join(wic_path, "train/train.data.txt")
wic_train_path_g = os.path.join(wic_path, "train/train.gold.txt")

train_data_df = pandas.read_csv(wic_train_path_d, sep = "\t", names = ['word', 'pos', 'position1', 'sent1', 'sent2'])
train_gold_df = pandas.read_csv(wic_train_path_g, sep = "\t", names = ['label'])

train_data_df[['position1', 'position2']] = train_data_df['position1'].str.split("-", expand = True)
train_data_df = train_data_df[['word', 'pos', 'position1', 'position2', 'sent1', 'sent2']]

train_data_df.head()
# train_gold_df.head()

,word,pos,position1,position2,sent1,sent2
0,carry,V,2,1,You must carry your camping gear .,Sound carries well over water .
1,go,V,2,6,Messages must go through diplomatic channels .,Do you think the sofa will go through the door ?
2,break,V,0,2,Break an alibi .,The wholesaler broke the container loads into ...
3,cup,N,8,4,He wore a jock strap with a metal cup .,Bees filled the waxen cups with honey .
4,academy,N,1,2,The Academy of Music .,The French Academy .


In [6]:
# do same for dev data
wic_dev_path_d = os.path.join(wic_path, "dev/dev.data.txt")
wic_dev_path_g = os.path.join(wic_path, "dev/dev.gold.txt")

dev_data_df = pandas.read_csv(wic_dev_path_d, sep = "\t", names = ['word', 'pos', 'position1', 'sent1', 'sent2'])
dev_gold_df = pandas.read_csv(wic_dev_path_g, sep = "\t", names = ['label'])
print(dev_data_df)

dev_data_df[['position1', 'position2']] = dev_data_df['position1'].str.split("-", expand = True)
dev_data_df = dev_data_df[['word', 'pos', 'position1', 'position2', 'sent1', 'sent2']]

dev_data_df.head()
# dev_gold_df.head()

            word  ...                                              sent2
0          board  ...              He nailed boards across the windows .
1      circulate  ...  This letter is being circulated among the facu...
2           hook  ...  He hooked a snake accidentally , and was so sc...
3     recreation  ...  Drug abuse is often regarded as a form of recr...
4    domesticity  ...  A royal family living in unpretentious domesti...
..           ...  ...                                                ...
633         base  ...  Bases include oxides and hydroxides of metals ...
634        power  ...         The mysterious presence of an evil power .
635  portmanteau  ...  ` motel ' is a portmanteau word made by combin...
636      promise  ...                      I promised somebody my time .
637       pierce  ...                      The path pierced the jungle .

[638 rows x 5 columns]


,word,pos,position1,position2,sent1,sent2
0,board,N,2,2,Room and board .,He nailed boards across the windows .
1,circulate,V,0,4,Circulate a rumor .,This letter is being circulated among the facu...
2,hook,V,0,1,Hook a fish .,"He hooked a snake accidentally , and was so sc..."
3,recreation,N,1,9,For recreation he wrote poetry and solved cros...,Drug abuse is often regarded as a form of recr...
4,domesticity,N,4,6,Making a hobby of domesticity .,A royal family living in unpretentious domesti...


In [7]:
!nvidia-smi

Mon Oct 19 17:29:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [9]:
!pip install transformers

     |████████████████████████████████| 1.1MB 7.2MB/s 
     |████████████████████████████████| 1.1MB 37.8MB/s 
     |████████████████████████████████| 890kB 39.8MB/s 
     |████████████████████████████████| 3.0MB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5fb069f00159df2065102a1dbcba3ebc9b745204aff69e4bf1b93969118730de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:
from transformers import BertTokenizerFast

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [11]:
# l = int(len(train_data_df['sent1'])*0.8)

In [12]:
# sentences is a list 0f str
train_sentences_1 =  list(train_data_df['sent1'])
train_sentences_2 = list(train_data_df['sent2'])
print(len(train_sentences_1))

dev_sentences_1 =  list(dev_data_df['sent1'])
dev_sentences_2 = list(dev_data_df['sent2'])
print(len(dev_sentences_1))

# test_sentences_1 =  list(test_data_df['sent1'])
# test_sentences_2 = list(test_data_df['sent2'])
# print(len(test_sentences_1))

# dev_sentences_1 =  list(train_data_df['sent1'])[l:]
# dev_sentences_2 = list(train_data_df['sent2'])[l:]
# print(len(dev_sentences_1))

5428
638


In [13]:
# find out the offset start position for each character
train_offsets_1 = []
train_offsets_2 = []

for i in range(len(train_sentences_1)):
  sentence_1 = train_sentences_1[i]
  sentence_2 = train_sentences_2[i]

  sentence_1 = sentence_1.split()
  offset_1 = 0
  for j, word in enumerate(sentence_1):
    if j == int(train_data_df['position1'][i]):
      break
    offset_1 = offset_1 + len(word)+1
  train_offsets_1.append(offset_1)

  sentence_2 = sentence_2.split()
  offset_2 = 0
  for j, word in enumerate(sentence_2):
    if j == int(train_data_df['position2'][i]):
      break
    offset_2 = offset_2 + len(word)+1
  train_offsets_2.append(offset_2)

print(len(train_sentences_2))
print(len(train_offsets_2))

5428
5428


In [14]:
encoded_inputs_train1 = tokenizer(train_sentences_1, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
encoded_inputs_train2 = tokenizer(train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)


encoded_inputs_dev1 = tokenizer(dev_sentences_1, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
encoded_inputs_dev2 = tokenizer(dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
# print(len(encoded_inputs_dev['input_ids'][0]))
# print(type(encoded_inputs_dev['input_ids']))

# encoded_inputs_test = tokenizer(test_sentences_1, test_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

In [15]:
# create wordpeice indices of the words of interest now
train_pos_1 = []
train_pos_2 = []
for j in range(len(train_sentences_1)):
  vocab_ind = encoded_inputs_train1['input_ids'][j].detach().numpy()
  which_word = 0
  pos1 = -1
  target_word_1 = int(train_data_df['position1'][j])
  # print("The targets are "+str((target_word_1, target_word_2)) + " and we have j = "+str(j))
  for i,v_ind in enumerate(vocab_ind):
    # print(v_ind)
    # skip cls token
    if i == 0:
      # print("oui"+ str(v_ind))
      which_word = 0
      old_char_poses =  encoded_inputs_train1['offset_mapping'][j][i].detach().numpy()
      continue

    # this token's character positions
    char_poses =  encoded_inputs_train1['offset_mapping'][j][i].detach().numpy()
    # print(char_poses)

    if char_poses[0] == old_char_poses[1]+1:
      which_word = which_word + 1

    if (which_word == target_word_1):
      done = True
      pos1 = i
      break

    old_char_poses = char_poses

  train_pos_1.append(pos1)

for j in range(len(train_sentences_2)):
  vocab_ind = encoded_inputs_train2['input_ids'][j].detach().numpy()
  which_word = 0
  pos2 = -1
  target_word_2 = int(train_data_df['position2'][j])
  # print("The targets are "+str((target_word_1, target_word_2)) + " and we have j = "+str(j))
  for i,v_ind in enumerate(vocab_ind):
    # skip cls token
    if i == 0:
      # print("oui"+ str(v_ind))
      which_word = 0
      old_char_poses =  encoded_inputs_train2['offset_mapping'][j][i].detach().numpy()
      continue

    # this token's character positions
    char_poses =  encoded_inputs_train2['offset_mapping'][j][i].detach().numpy()
    # print(char_poses)

    if char_poses[0] == old_char_poses[1]+1:
      which_word = which_word + 1

    if (which_word == target_word_2):
      done = True
      pos2 = i
      break

    old_char_poses = char_poses

  train_pos_2.append(pos2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  vocab_ind = encoded_inputs_dev1['input_ids'][j].detach().numpy()
  which_word = 0
  pos1 = -1
  target_word_1 = int(dev_data_df['position1'][j])
  # print("The targets are "+str((target_word_1, target_word_2)) + " and we have j = "+str(j))
  for i,v_ind in enumerate(vocab_ind):
    # print(v_ind)
    # skip cls token
    if i == 0:
      # print("oui"+ str(v_ind))
      which_word = 0
      old_char_poses =  encoded_inputs_dev1['offset_mapping'][j][i].detach().numpy()
      continue

    # this token's character positions
    char_poses =  encoded_inputs_dev1['offset_mapping'][j][i].detach().numpy()
    # print(char_poses)

    if char_poses[0] == old_char_poses[1]+1:
      which_word = which_word + 1

    if (which_word == target_word_1):
      done = True
      pos1 = i
      break

    old_char_poses = char_poses

  dev_pos_1.append(pos1)


for j in range(len(dev_sentences_2)):
  vocab_ind = encoded_inputs_dev1['input_ids'][j].detach().numpy()
  which_word = 0
  pos2 = -1
  target_word_2 = int(dev_data_df['position2'][j])
  # print("The targets are "+str((target_word_1, target_word_2)) + " and we have j = "+str(j))
  for i,v_ind in enumerate(vocab_ind):
    # print(v_ind)
    # skip cls token
    if i == 0:
      # print("oui"+ str(v_ind))
      which_word = 0
      old_char_poses =  encoded_inputs_dev2['offset_mapping'][j][i].detach().numpy()
      continue

    # this token's character positions
    char_poses =  encoded_inputs_dev2['offset_mapping'][j][i].detach().numpy()
    # print(char_poses)

    if char_poses[0] == old_char_poses[1]+1:
      which_word = which_word + 1

    if (which_word == target_word_2):
      done = True
      pos2 = i
      break

    old_char_poses = char_poses

  dev_pos_2.append(pos2)

print(len(train_pos_1))
print(len(dev_pos_2))

train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)
print((train_pos.size()))

# print(dev_pos[:20] - train_pos[:20])


5428
638
torch.Size([5428, 2])


In [16]:
# labels = torch.from_numpy(train_gold_df['label'].values)
train_gold_df_tmp = train_gold_df.replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)
print(train_labels)

dev_gold_df_tmp = dev_gold_df.replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)
print(len(dev_labels))

# test_gold_df_tmp = test_gold_df.replace({'F' : 0, 'T' : 1})
# test_labels = torch.from_numpy(test_gold_df_tmp.values)
# print(len(test_labels))

tensor([[0],
        [0],
        [0],
        ...,
        [1],
        [1],
        [1]])
638


In [47]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train1['input_ids'], encoded_inputs_train1['token_type_ids'],
                              encoded_inputs_train1['attention_mask'], encoded_inputs_train2['input_ids'], encoded_inputs_train2['token_type_ids'],
                              encoded_inputs_train2['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev1['input_ids'], encoded_inputs_dev1['token_type_ids'],
                              encoded_inputs_dev1['attention_mask'], encoded_inputs_dev2['input_ids'], encoded_inputs_dev2['token_type_ids'],
                              encoded_inputs_dev2['attention_mask'], dev_pos, dev_labels)

# test_dataset = TensorDataset(encoded_inputs_test['input_ids'], encoded_inputs_test['token_type_ids'],
#                               encoded_inputs_test['attention_mask'], test_pos, test_labels)


In [48]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            batch_size = batch_size # Evaluate with this batch size.
        )

# test_dataloader = DataLoader(
#             test_dataset, # The validation samples.
#             sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
#             batch_size = batch_size # Evaluate with this batch size.
#         )

**CUTOFF**

In [19]:
from transformers import BertModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "bert-base-uncased"
        hidden_states = True
        self.encoder = BertModel.from_pretrained(options_name, output_hidden_states = hidden_states)

    def forward(self, input_ids, token_type_ids, attention_mask):
        last_layer, _ , hidden_states= self.encoder(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)
        second_last_layer = hidden_states[-2]
        return second_last_layer

In [20]:
model_bert = BERTi().to(device)
# Get all of the model's parameters as a list of tuples.
params = list(model_bert.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


The BERT model has 199 different named parameters.

==== Embedding Layer ====

encoder.embeddings.word_embeddings.weight               (30522, 768)
encoder.embeddings.position_embeddings.weight             (512, 768)
encoder.embeddings.token_type_embeddings.weight             (2, 768)
encoder.embeddings.LayerNorm.weight                           (768,)
encoder.embeddings.LayerNorm.bias                             (768,)

==== First Transformer ====

encoder.encoder.layer.0.attention.self.query.weight       (768, 768)
encoder.encoder.layer.0.attention.self.query.bias             (768,)
encoder.encoder.layer.0.attention.self.key.weight         (768, 768)
encoder.encoder.layer.0.attention.self.key.bias               (768,)
encoder.encoder.layer.0.attention.self.value.weight       (768, 768)
encoder.encoder.layer.0.attention.self.value.bias             (768,)
encoder.encoder.layer.0.attention.output.dense.weight     (768, 768)
encoder.encoder.layer.0.attention.output.dense.bias           

In [21]:
print(type(model_bert.parameters()))

<class 'generator'>


In [22]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    print(pred_flat)
    print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [23]:
def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

t0 = time.time()
# N x 1
similarities = []
labels = []
with torch.no_grad():
  for step, batch in enumerate(train_dataloader):


          # Progress update every 40 batches.
          if step % 50 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))



          b_input_ids1 = batch[0].to(device)
          b_token_type_ids1 = batch[1].to(device)
          b_attention_mask1 = batch[2].to(device)
          b_input_ids2 = batch[3].to(device)
          b_token_type_ids2 = batch[4].to(device)
          b_attention_mask2 = batch[5].to(device)
          b_poses = batch[6].to(device)
          b_labels = batch[7].to(device)
      

          second_last_layer1 = model_bert(b_input_ids1, b_token_type_ids1, b_attention_mask1)

          second_last_layer2 = model_bert(b_input_ids2, b_token_type_ids2, b_attention_mask2)

          # print(second_last_layer1.size())
          b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
          # print(b_poses[:,:1,:].size())
          gathered_activations1 = torch.gather(second_last_layer1, 1, b_poses[:,:1,:])
          gathered_activations2 = torch.gather(second_last_layer2, 1, b_poses[:,1:2,:])
          # print(gathered_activations2.squeeze(dim = 1).size())

          similarity = cosine_similarity(gathered_activations1.squeeze(dim = 1).cpu(), gathered_activations2.squeeze(dim = 1).cpu()).diagonal().reshape(b_labels.size()[0], -1)
          similarities.append(similarity)
          # # VVVV IMPORTANT AS BATCHES HAVE BEEN SHUFFLED WHICH WAS NOT NECESSSARY - SAVE FILE IS USELESS
          # labels.append(b_labels)
          # print(similarity)
          # print(b_labels)
          # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

similarities_np = np.concatenate(similarities, axis = 0)


  Batch    50  of    340.    Elapsed: 0:00:09.
  Batch   100  of    340.    Elapsed: 0:00:17.
  Batch   150  of    340.    Elapsed: 0:00:26.
  Batch   200  of    340.    Elapsed: 0:00:34.
  Batch   250  of    340.    Elapsed: 0:00:43.
  Batch   300  of    340.    Elapsed: 0:00:51.


In [50]:
np.save("/content/drive/My Drive/datasets/Split_WiC_dataset/Cosine_Sim/numpy_file_cos_sim", similarities_np)

In [51]:
def thresholded_accuracy(labels, preds, threshold):
  preds_th = preds.copy()
  preds_th[preds < threshold] = 0
  preds_th[~(preds < threshold)] = 1

  correct = (preds_th == labels).sum()
  # print(correct)
  return correct/preds.shape[0]

In [63]:
# implement linear search
train_labels_np = train_labels.detach().numpy()

steps = 1000
step_size = 1/steps

best_accuracy = 0.0
best_threshold = 0.0
for i in range(steps+1):

  threshold = i * step_size
  # print(threshold)
  accuracy = thresholded_accuracy(train_labels_np, similarities_np, threshold)

  # print(accuracy)
  if (accuracy > best_accuracy):
    best_accuracy = accuracy
    best_threshold = threshold

print("Best Accuracy = "+str(best_accuracy))
print("Threshold = "+str(best_threshold))
# print(similarities_np[:100])

Best Accuracy = 0.6836772291820191
Threshold = 0.584


In [64]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

t0 = time.time()
# N x 1
similarities_test = []
with torch.no_grad():
  for step, batch in enumerate(dev_dataloader):


          # Progress update every 40 batches.
          if step % 50 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))



          b_input_ids1 = batch[0].to(device)
          b_token_type_ids1 = batch[1].to(device)
          b_attention_mask1 = batch[2].to(device)
          b_input_ids2 = batch[3].to(device)
          b_token_type_ids2 = batch[4].to(device)
          b_attention_mask2 = batch[5].to(device)
          b_poses = batch[6].to(device)
          b_labels = batch[7].to(device)
      

          second_last_layer1 = model_bert(b_input_ids1, b_token_type_ids1, b_attention_mask1)

          second_last_layer2 = model_bert(b_input_ids2, b_token_type_ids2, b_attention_mask2)

          # print(second_last_layer1.size())
          b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
          # print(b_poses[:,:1,:].size())
          gathered_activations1 = torch.gather(second_last_layer1, 1, b_poses[:,:1,:])
          gathered_activations2 = torch.gather(second_last_layer2, 1, b_poses[:,1:2,:])
          # print(gathered_activations2.squeeze(dim = 1).size())

          similarity = cosine_similarity(gathered_activations1.squeeze(dim = 1).cpu(), gathered_activations2.squeeze(dim = 1).cpu()).diagonal().reshape(b_labels.size()[0], -1)
          similarities_test.append(similarity)
          # # VVVV IMPORTANT AS BATCHES HAVE BEEN SHUFFLED WHICH WAS NOT NECESSSARY - SAVE FILE IS USELESS
          # labels.append(b_labels)
          # print(similarity)
          # print(b_labels)
          # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

similarities_test_np = np.concatenate(similarities_test, axis = 0)

In [65]:
# Test
dev_labels_np = dev_labels.detach().numpy()

test_accuracy = thresholded_accuracy(dev_labels_np, similarities_test_np, best_threshold)

print("The test accuracy with best threshold = "+str(best_threshold)+" is = "+str(test_accuracy))

The test accuracy with best threshold = 0.584 is = 0.6489028213166145
